# Data Cleaning & Augmentation

## Simplify chords

Simplifying chords down to 42: base note (A-G) + accidental + major/minor(dim).

In [4]:
import pandas as pd
import numpy as np
import json
import csv
from sklearn.feature_extraction.text import CountVectorizer

In [40]:
### Simplifying chords down to set of 42 ###
notes = ["A", "B", "C", "D", "E", "F", "G"]
accs = ["b", "s", ""]
all_notes_list = [note + acc for note in notes for acc in accs]

def simplify_chord(chord: str) -> str:
    """
    Removes chord quality from a chord.
    """
    for note in all_notes_list:
        if not chord.startswith(note):
            continue

        suffix = chord.removeprefix(note)
        if suffix.startswith("min") or suffix.startswith("dim"):
            return note + "min"
        else:
            return note

    if chord == "sC":
        return "Cs"

    # print(chord)
    return ""

In [41]:
df = pd.read_csv("chordonomicon_v2.csv", usecols=["id", "chords", "main_genre"])
df["chords"] = (
    df["chords"]
    .str.split(" ")
    .map(lambda lst: [simplify_chord(x) for x in lst if not x.startswith("<")])
)
df.to_csv("chordonomicon_v2_simplified.csv", index=False)

/var/folders/pr/6c9tvpf916nbbt_5rlv6rcxr0000gn/T/ipykernel_8455/3360250008.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("chordonomicon_v2.csv", usecols=["id", "chords", "main_genre"])


Optionally, also collaps both sharp and flat accents to sharp:

In [42]:
def simplify_and_standardize(chord: str) -> str:
    """
    Simplifies chord quality (min/dim) and standardizes flat prefixes.
    """
    # --- 1) Simplification (your original simplify_chord) ---
    simplified = ""
    for note in all_notes_list:
        if chord.startswith(note):
            suffix = chord[len(note):]
            if suffix.startswith("min") or suffix.startswith("dim"):
                simplified = note + "min"
            else:
                simplified = note
            break

    if chord == "sC":
        simplified = "Cs"

    if simplified == "":
        simplified = chord  # fallback to original if nothing matched

    # --- 2) Standardization (your standardize_chord_prefix) ---
    flat_to_sharp = {
        "Bb": "As",
        "Db": "Cs",
        "Eb": "Ds",
        "Gb": "Fs",
        "Ab": "Gs",
        "Cb": "B",
        "Fb": "E",
        "Bs": "C",
        "Es": "F"
    }

    for flat, sharp in flat_to_sharp.items():
        if simplified.startswith(flat):
            return sharp + simplified[len(flat):]

    return simplified


In [43]:
df = pd.read_csv("chordonomicon_v2.csv", usecols=["id", "chords", "main_genre"])
df["chords"] = (
    df["chords"]
    .str.split(" ")
    .map(lambda lst: [simplify_and_standardize(x) for x in lst if not x.startswith("<")])
)
df.to_csv("chordonomicon_v2_standardized.csv", index=False)

/var/folders/pr/6c9tvpf916nbbt_5rlv6rcxr0000gn/T/ipykernel_8455/3026761472.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("chordonomicon_v2.csv", usecols=["id", "chords", "main_genre"])


## Transpose to all chords
Transpose each song to mitigate bias from the key of the song and to augment our data. Also Collapse both sharp and flat accents to sharp

In [ ]:
### Transposing songs to all keys ###
keys_list = ["C", "Cs", "D", "Ds", "E", "F", "Fs", "G", "Gs", "A", "As", "B"]

def transpose_chord(chord: str, variation: int) -> str:
    # Identify base note and suffix robustly
    if len(chord) >= 2 and chord[1] == "s":
        base = chord[:2]
        suffix = chord[2:]
    else:
        base = chord[:1]
        suffix = chord[1:]
    if base in keys_list:
        idx = keys_list.index(base)
        new_base = keys_list[(idx + variation) % 12]
        return new_base + suffix
    return chord  # If not found, return as is

def augment_keys(df):
    augmented_rows = []
    for _, row in df.iterrows():
        for variation in range(12):
            new_row = row.copy()
            if variation == 0:
                new_row["original_key"] = True
            else:
                new_row["original_key"] = False
            new_row["added_semitones"] = variation
            new_row["chords"] = [transpose_chord(chord, variation) for chord in row["chords"]]
            augmented_rows.append(new_row)
    return pd.DataFrame(augmented_rows)

def standardize_chord_prefix(chord: str) -> str:
    # Map flat notes to their sharp equivalents
    flat_to_sharp = {
        "Bb": "As",
        "Db": "Cs",
        "Eb": "Ds",
        "Gb": "Fs",
        "Ab": "Gs",
        "Cb": "B",
        "Fb": "E",
        "Bs": "C",
        "Es": "F"
    }
    for flat, sharp in flat_to_sharp.items():
        if chord.startswith(flat):
            return sharp + chord[len(flat):]
    return chord

In [4]:
# Writing transposed dataset to csv
def get_pop_chords_df():
    df = pd.read_csv("hf://datasets/ailsntua/Chordonomicon/chordonomicon_v2.csv",usecols=["id", "chords", "main_genre"])
    pop_df = df[df["main_genre"] == "pop"][["id", "chords"]].copy()
    pop_df["chords"] = pop_df["chords"].str.split(" ")
    pop_df["chords"] = pop_df["chords"].map(
        lambda chords: [simplify_chord(standardize_chord_prefix(chord)) for chord in chords if not chord.startswith("<")]
    )

    pop_df["original_key"] = True
    pop_df["added_semitones"] = 0
    return pop_df

pop_chords_df = get_pop_chords_df()
augmented_df = augment_keys(pop_chords_df)
print(augmented_df.head(15))
augmented_df.to_csv("chordonomicon_v2_augmented.csv", index=False)

/Users/sergimarsol/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/pr/6c9tvpf916nbbt_5rlv6rcxr0000gn/T/ipykernel_8455/484076371.py:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("hf://datasets/ailsntua/Chordonomicon/chordonomicon_v2.csv",usecols=["id", "chords", "main_genre"])


   id                                             chords  original_key  \
0   1  [C, F, C, E, Amin, C, F, C, G, C, F, C, E, Ami...          True   
0   1  [Cs, Fs, Cs, F, Asmin, Cs, Fs, Cs, Gs, Cs, Fs,...         False   
0   1  [D, G, D, Fs, Bmin, D, G, D, A, D, G, D, Fs, B...         False   
0   1  [Ds, Gs, Ds, G, Cmin, Ds, Gs, Ds, As, Ds, Gs, ...         False   
0   1  [E, A, E, Gs, Csmin, E, A, E, B, E, A, E, Gs, ...         False   
0   1  [F, As, F, A, Dmin, F, As, F, C, F, As, F, A, ...         False   
0   1  [Fs, B, Fs, As, Dsmin, Fs, B, Fs, Cs, Fs, B, F...         False   
0   1  [G, C, G, B, Emin, G, C, G, D, G, C, G, B, Emi...         False   
0   1  [Gs, Cs, Gs, C, Fmin, Gs, Cs, Gs, Ds, Gs, Cs, ...         False   
0   1  [A, D, A, Cs, Fsmin, A, D, A, E, A, D, A, Cs, ...         False   
0   1  [As, Ds, As, D, Gmin, As, Ds, As, F, As, Ds, A...         False   
0   1  [B, E, B, Ds, Gsmin, B, E, B, Fs, B, E, B, Ds,...         False   
4   5  [C, G, C, G, C, F, Dmin, G, Dmi

# N-gram Learning
Compute n-gram counts using CountVectorizer library (usually used for bag of n-grams)

In [45]:
def count_n_grams(data, n: int = 1) -> pd.DataFrame:
    word_vectorizer = CountVectorizer(
        ngram_range=(1, n),
        analyzer="word",
        token_pattern=r"(?u)\b\w+\b",
        lowercase=False,
    )

    sparse_matrix = word_vectorizer.fit_transform(
        data.map(lambda chords: " ".join(chords))
    )

    frequencies = sum(sparse_matrix).toarray()[0]

    df_all = pd.DataFrame(
        frequencies,
        index=word_vectorizer.get_feature_names_out(),
        columns=["count"],
    )

    return df_all.groupby(by=lambda chords: len(chords.split(" ")))

Import our processed dataset and compute n-gram counts.

In [ ]:
import pandas as pd
import ast
import json

def load_chords_from_csv(path):
    df = pd.read_csv(path, usecols=['chords'])
    # If chords are stored as a list string, use ast.literal_eval
    try:
        chords = df['chords'].apply(ast.literal_eval)
    except Exception:
        chords = df['chords'].str.split(" ")
    return chords

results = {}

for name, path in [
    ("simplified", "chordonomicon_v2_simplified.csv"),
    ("standardized", "chordonomicon_v2_standardized.csv"),
    ("augmented", "chordonomicon_v2_augmented.csv")
]:
    chords = load_chords_from_csv(path)
    n_gram_counts = count_n_grams(chords, 3)
    results[name] = {}
    for key, _ in n_gram_counts:
        df = n_gram_counts.get_group(key).sort_values(by='count')
        results[name][f"{key}-gram"] = df.reset_index().to_dict(orient="records")
        if name == "simplified":
            print(f"\n--- {key}-gram counts for simplified ---")
            print(df)

with open("n_gram_counts_results.json", "w") as f:
    json.dump(results, f, indent=2)


--- 1-gram counts for simplified ---
         count
Gbmin    13921
Dbmin    25731
Bs       43297
Abmin    50386
Es       53824
Asmin    76742
Ebmin    98737
Gb      115516
Dsmin   136273
Bbmin   173213
Db      226807
Fmin    349631
As      361219
Gsmin   365534
Ds      370729
Gs      379959
Ab      415987
Cs      458960
Cmin    485114
Eb      583180
Gmin    656014
Csmin   700142
Fs      842932
Fsmin   991863
Bb     1147575
Dmin   1449889
Bmin   1633942
B      1760255
Emin   2989466
E      3172313
Amin   3277296
F      3718124
A      4527730
D      6005289
C      6627164
G      7709880

--- 2-gram counts for simplified ---
               count
Asmin Gbmin        1
Gsmin Gbmin        1
Dsmin Gbmin        1
Dbmin Dsmin        1
Dsmin Dbmin        2
...              ...
F C          1275024
D G          1604703
G D          1747895
G C          1889671
C G          2208974

[1293 rows x 1 columns]

--- 3-gram counts for simplified ---
                   count
Dsmin Ebmin Gs         1
Fsmi

Calculate transition matrix probabilities using counts.

In [105]:
import itertools

# base notes and shape of chord names
notes = ["A", "B", "C", "D", "E", "F", "G"]
# standardized/augmented: no flat spellings (only natural + sharp 's')
accs_std = ["", "s"]
# simplified: include flats 'b' in addition to sharps and naturals
accs_simpl = ["", "s", "b"]
third = ["", "min"]

# exclude weird/duplicate forms for standardized (same as you used)
excluded_chords = {"Bs", "Bsmin", "Es", "Esmin", "Cb", "Cbmin", "Fb", "Fbmin"}

# build standardized / augmented list
all_chords_std = [note + acc + t for note in notes for acc in accs_std for t in third]
all_chords_std = [c for c in all_chords_std if c not in excluded_chords]

# build simplified list (includes flats)
all_chords_simpl = [note + acc + t for note in notes for acc in accs_simpl for t in third]
all_chords_simpl = [c for c in all_chords_simpl if c not in excluded_chords]

# final dict
all_chords_dict = {
    "standardized": sorted(all_chords_std),
    "augmented": sorted(all_chords_std),   # same as standardized
    "simplified": sorted(all_chords_simpl),
}

print("standardized (count):", len(all_chords_dict["standardized"]))
print(all_chords_dict["standardized"])
print("simplified (count):", len(all_chords_dict["simplified"]))
print(all_chords_dict["simplified"])

standardized (count): 24
['A', 'Amin', 'As', 'Asmin', 'B', 'Bmin', 'C', 'Cmin', 'Cs', 'Csmin', 'D', 'Dmin', 'Ds', 'Dsmin', 'E', 'Emin', 'F', 'Fmin', 'Fs', 'Fsmin', 'G', 'Gmin', 'Gs', 'Gsmin']
simplified (count): 34
['A', 'Ab', 'Abmin', 'Amin', 'As', 'Asmin', 'B', 'Bb', 'Bbmin', 'Bmin', 'C', 'Cmin', 'Cs', 'Csmin', 'D', 'Db', 'Dbmin', 'Dmin', 'Ds', 'Dsmin', 'E', 'Eb', 'Ebmin', 'Emin', 'F', 'Fmin', 'Fs', 'Fsmin', 'G', 'Gb', 'Gbmin', 'Gmin', 'Gs', 'Gsmin']


In [106]:
# Calculate transition matrix probabilities
# alpha is additive smoothing

def compute_unigram_prob(n_gram_counts, alpha=1.0, input_data="simplified"):
    unigram = n_gram_counts.get_group(1)
    unigram = unigram.reindex(all_chords_dict[input_data], fill_value=0)
    vocab_size = len(all_chords_dict[input_data])
    total_count = unigram["count"].sum()

    probs = (unigram["count"] + alpha) / (total_count + alpha * vocab_size)
    df = pd.DataFrame([probs.values], 
                      index=[""],
                      columns=all_chords_dict[input_data])
    return df

In [107]:
def compute_unigram_prob_from_df(unigram_df: pd.DataFrame, alpha: float = 1.0, input_data: str = "simplified") -> pd.DataFrame:
    """
    Compute unigram probabilities with additive smoothing directly from a DataFrame.
    
    Parameters:
    - unigram_df: DataFrame containing columns ['index', 'count', ...] for 1-grams
    - alpha: additive smoothing constant
    - input_data: key to select chord vocabulary from all_chords_dict
    
    Returns:
    - DataFrame: 1-row, columns = chords, values = probability
    """
    # Set index to chord names
    unigram_df = unigram_df.set_index("index")

    # Reindex to include all possible chords, fill missing with 0
    vocab = all_chords_dict[input_data]
    unigram_df = unigram_df.reindex(vocab, fill_value=0)

    # Compute probabilities with additive smoothing
    total_count = unigram_df["count"].sum()
    probs = (unigram_df["count"] + alpha) / (total_count + alpha * len(vocab))

    # Return as a 1-row DataFrame with chords as columns
    return pd.DataFrame([probs.values], columns=vocab, index=[""])


In [108]:
# Extract unigrams from df_all
unigram_df = df_all[df_all["ngram_length"] == 1].copy()

# Compute probabilities
unigram_probs = compute_unigram_prob_from_df(unigram_df, alpha=1.0, input_data="simplified")

print(unigram_probs)


         A        Ab     Abmin      Amin       As     Asmin         B  \
  0.087244  0.008016  0.000971  0.063149  0.00696  0.001479  0.033918   

        Bb     Bbmin      Bmin  ...         F      Fmin        Fs     Fsmin  \
  0.022112  0.003338  0.031484  ...  0.071644  0.006737  0.016242  0.019112   

        G        Gb     Gbmin      Gmin        Gs     Gsmin  
  0.14856  0.002226  0.000268  0.012641  0.007321  0.007043  

[1 rows x 34 columns]


In [109]:
import itertools
import pandas as pd

def compute_ngram_prob_from_df(ngram_df, n: int = 2, alpha=1.0, input_data="simplified"):
    """
    Compute n-gram probabilities from a reconstructed DataFrame.
    ngram_df: DataFrame with columns ['index', 'count', 'ngram_length'] for n-grams
    """
    ngram = ngram_df.copy()

    # use the 'index' column as the n-gram string
    ngram["evidence"] = ngram["index"].map(lambda s: " ".join(s.split()[:-1]))
    ngram["next"] = ngram["index"].map(lambda s: s.split()[-1])

    # generate all possible (n-1)-length sequences
    all_evidence_seq = [" ".join(evidence) for evidence in itertools.product(all_chords_dict[input_data], repeat=(n - 1))]
    full_index = pd.MultiIndex.from_product([all_evidence_seq, all_chords_dict[input_data]], names=["evidence", "next"])

    # reindex to include all possible n-grams
    ngram = ngram.set_index(["evidence", "next"])
    ngram = ngram.reindex(full_index, fill_value=0)

    # compute probabilities with additive smoothing
    evidence_counts = ngram["count"].groupby(level="evidence").transform("sum")
    vocab_size = len(all_chords_dict[input_data])
    ngram["prob"] = (ngram["count"] + alpha) / (evidence_counts + alpha * vocab_size)

    # return as a 2D DataFrame: rows=evidence, columns=next chord
    return ngram["prob"].unstack(fill_value=0.0)


In [ ]:
bigram_df = df_all[df_all["ngram_length"] == 2].copy()
bigram_probs = compute_ngram_prob_from_df(bigram_df, n=2, alpha=1.0, input_data="simplified")
print(bigram_probs.shape)
bigram_probs


(34, 34)


next,A,Ab,Abmin,Amin,As,Asmin,B,Bb,Bbmin,Bmin,...,F,Fmin,Fs,Fsmin,G,Gb,Gbmin,Gmin,Gs,Gsmin
evidence,,,,,,,,,,,,,,,,,,,,,
A,0.032495,0.001423,0.000442,0.008960,0.011107,0.000477,0.070782,0.006218,0.000392,0.080800,...,0.011444,0.000757,0.011150,0.047021,0.117500,0.000276,0.000425,0.002431,0.005924,0.006417
Ab,0.012838,0.027735,0.008279,0.004620,0.000871,0.000144,0.011306,0.148031,0.059936,0.001744,...,0.024407,0.078073,0.008466,0.001151,0.047344,0.041855,0.000771,0.030104,0.001253,0.000297
Abmin,0.061594,0.019923,0.018019,0.018220,0.000842,0.000020,0.097853,0.039025,0.048486,0.006755,...,0.006033,0.012768,0.056623,0.017919,0.015895,0.061534,0.011525,0.023411,0.000200,0.000521
Amin,0.007785,0.001025,0.000332,0.024451,0.005956,0.000213,0.015264,0.015467,0.000479,0.018030,...,0.206059,0.001789,0.001440,0.001384,0.204272,0.000057,0.000049,0.008746,0.002628,0.000863
As,0.178235,0.000814,0.000031,0.043651,0.021968,0.003697,0.031293,0.002155,0.000273,0.022053,...,0.105619,0.006623,0.013176,0.011109,0.056562,0.000017,0.000080,0.028386,0.037045,0.004204
Asmin,0.014917,0.000961,0.000053,0.017971,0.007162,0.009716,0.102429,0.000263,0.000382,0.028438,...,0.027398,0.048397,0.173287,0.007188,0.006096,0.000158,0.000026,0.005398,0.145033,0.046883
B,0.151144,0.001889,0.003277,0.011558,0.006679,0.002626,0.020622,0.006975,0.002224,0.006526,...,0.006046,0.000878,0.094461,0.027984,0.024680,0.005143,0.000325,0.001105,0.008157,0.034793
Bb,0.038821,0.041206,0.000747,0.029596,0.000958,0.000048,0.007322,0.023007,0.008243,0.003516,...,0.258419,0.015718,0.003335,0.000552,0.029415,0.002499,0.000077,0.069741,0.006359,0.000701
Bbmin,0.010587,0.132615,0.012489,0.020685,0.000408,0.000023,0.032206,0.012892,0.019851,0.011002,...,0.071831,0.096518,0.040367,0.001767,0.005302,0.098601,0.002065,0.007612,0.024757,0.004369


In [111]:
trigram_df = df_all[df_all["ngram_length"] == 3].copy()
trigram_probs = compute_ngram_prob_from_df(trigram_df, n=3, alpha=1.0, input_data="simplified")
print(trigram_probs.shape)
trigram_probs

(1156, 34)


next,A,Ab,Abmin,Amin,As,Asmin,B,Bb,Bbmin,Bmin,...,F,Fmin,Fs,Fsmin,G,Gb,Gbmin,Gmin,Gs,Gsmin
evidence,,,,,,,,,,,,,,,,,,,,,
A A,0.309128,0.000889,0.000245,0.007675,0.011106,0.000588,0.021007,0.003963,0.000371,0.038793,...,0.009957,0.000756,0.008466,0.049794,0.042694,0.000140,0.000238,0.002262,0.002759,0.003193
A Ab,0.181919,0.015279,0.011459,0.002865,0.000159,0.000159,0.040427,0.026580,0.007799,0.005889,...,0.016712,0.007640,0.036129,0.029126,0.241604,0.034538,0.006685,0.029445,0.004138,0.001910
A Abmin,0.176768,0.018182,0.004545,0.008586,0.001010,0.000505,0.086364,0.004545,0.004040,0.009091,...,0.004545,0.004040,0.023232,0.187879,0.033838,0.025758,0.080808,0.024747,0.000505,0.000505
A Amin,0.111784,0.001247,0.002138,0.024612,0.007432,0.000153,0.033342,0.003920,0.000458,0.024459,...,0.038330,0.000713,0.003309,0.011402,0.102163,0.000127,0.000255,0.004683,0.003614,0.016824
A As,0.573790,0.000205,0.000020,0.009450,0.025323,0.001534,0.038291,0.000777,0.000184,0.027348,...,0.025036,0.000614,0.003784,0.017407,0.045020,0.000020,0.000061,0.006198,0.002536,0.000859
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Gsmin Gb,0.006623,0.006623,0.006623,0.006623,0.033113,0.006623,0.132450,0.019868,0.006623,0.006623,...,0.006623,0.019868,0.006623,0.006623,0.006623,0.006623,0.006623,0.006623,0.006623,0.152318
Gsmin Gbmin,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,...,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571
Gsmin Gmin,0.030525,0.000407,0.000407,0.032967,0.002849,0.004477,0.015873,0.001221,0.002035,0.007733,...,0.034188,0.027676,0.021164,0.487586,0.006105,0.001221,0.000407,0.011803,0.001628,0.070818


# Inference
Deterministic and probabilistic methods, we are generating a 16 chord song simply from the log-probabilities from our models.

In [112]:
def deterministic_inference(evidence):
    # evidence: string of n-1 space-separated chords
    
    n = len(evidence.split()) + 1
    ngram_probs = unigram_probs if n == 1 else (bigram_probs if n == 2 else trigram_probs)

    if evidence not in ngram_probs.index:
        raise KeyError(f"Evidence '{evidence}' not found in {n}-gram table")
    
    row_probs = ngram_probs.loc[evidence]
    return row_probs.idxmax() # returns next chord w highest prob, if there are several, the first one in col order

def probabilistic_inference(evidence):
    # evidence: string of n-1 space-separated chords
    
    n = len(evidence.split()) + 1
    ngram_probs = unigram_probs if n == 1 else (bigram_probs if n == 2 else trigram_probs)

    if evidence not in ngram_probs.index:
        raise KeyError(f"Evidence '{evidence}' not found in {n}-gram table")
    
    row_probs = ngram_probs.loc[evidence]
    cdf = np.cumsum(row_probs.values) # create cumulative distribution over next possible chord

    # sample over dist
    seed = np.random.random()
    idx = np.searchsorted(cdf, seed)
    
    return row_probs.index[idx] # return probabilistically chosen next chord

In [113]:
### test inference for bigram ###
seq = []

for _ in range(16):
    if len(seq) == 0:
        evidence = ""
    elif len(seq) == 1:
        evidence = seq[-1]
    else:
        evidence = " ".join(seq[-2:])

    next_chord = deterministic_inference(evidence) # can change to deterministic_inference()
    seq.append(next_chord)

print(seq)

['G', 'C', 'G', 'C', 'G', 'C', 'G', 'C', 'G', 'C', 'G', 'C', 'G', 'C', 'G', 'C']


In [114]:
### test inference for bigram ###
seq = []

for _ in range(16):
    if len(seq) == 0:
        evidence = ""
    elif len(seq) == 1:
        evidence = seq[-1]
    else:
        evidence = " ".join(seq[-2:])

    next_chord = probabilistic_inference(evidence) # can change to deterministic_inference()
    seq.append(next_chord)

print(seq)

['Fs', 'Gsmin', 'Cs', 'Fs', 'Cs', 'Fs', 'B', 'E', 'Csmin', 'A', 'Fsmin', 'E', 'D', 'Fsmin', 'Emin', 'A']


# Evaluation
Evaluate log-likelihood of an n-gram given a song (in this case the first song of the dataset)

In [115]:
def song_log_likelihood_ngram(song, n, ngram_probs):
    # song: list of chords in song
    # n: order of the n-gram model
    # ngram_probs: dict[context_tuple] -> dict[target] = P(target | context)
    # ex: trigram ngram_prob = dict[(chord1, chord2)] = {chord0:P,...,chordV:P}, dict[chord3] = P(chord3 | chord1, chord2)
    # vocab_size: 42 or 36?

    ll = 0.0
    if len(song) < n:
        return 0.0

    for t in range(n-1, len(song)):
        if n == 1:
            context = ""
        else:
            context = " ".join(song[t-(n-1):t])

        target = song[t]

        try:
            p = ngram_probs.loc[context, target]
        except KeyError:
            print(f"KeyError given evidence {context}")
            p = 1e-12

        if p <= 0:
            p = 1e-12
        
        ll += np.log(p)

    return ll

In [116]:
# test log-likelihood
song_simplified = ['C', 'F', 'C', 'E', 'Amin', 'C', 'F', 'C', 'G', 'C', 'F', 'C', 'E', 'Amin', 'C', 'F', 'G', 'C', 'F', 'C', 'E', 'Amin', 'C', 'F', 'C', 'G', 'C', 'F', 'C', 'E', 'Amin', 'C', 'F', 'G', 'C', 'F', 'C', 'F', 'C', 'G', 'C', 'F', 'C', 'E', 'Amin', 'C', 'F', 'G', 'C', 'D', 'G', 'D', 'G', 'D', 'A', 'D', 'G', 'D', 'Fs', 'Bmin', 'D', 'G', 'A', 'D', 'G', 'A', 'D']
print(song_log_likelihood_ngram(song_simplified, 1, unigram_probs))
print(song_log_likelihood_ngram(song_simplified, 2, bigram_probs))
print(song_log_likelihood_ngram(song_simplified, 3, trigram_probs))

-155.76098994173117
-121.75005315476847
-103.20723142130456


Also compute the top-k accuracy with k = 1,3,5 for the given song

In [117]:
def top_k_accuracy_ngram(song, n, ngram_probs, k=5):
    # song: list of chords in song
    # n: order of the n-gram model
    # ngram_probs: DataFrame with index=evidence, columns=next chords
    # k: number of top predictions to consider
    
    correct = 0
    total = 0
    
    if len(song) < n:
        return 0.0
    
    for t in range(n-1, len(song)):
        if n == 1:
            context = ""
        else:
            context = " ".join(song[t-(n-1):t])
        
        target = song[t]
        
        try:
            prob_row = ngram_probs.loc[context]
            
            top_k_chords = prob_row.nlargest(k).index.tolist()
            
            if target in top_k_chords:
                correct += 1
            total += 1
            
        except KeyError:
            total += 1
    
    return correct / total if total > 0 else 0.0

In [118]:
# Test on top-k accuracy
print(f"Top-1: {top_k_accuracy_ngram(seq, 2, bigram_probs, k=1):.4f}")
print(f"Top-3: {top_k_accuracy_ngram(seq, 2, bigram_probs, k=3):.4f}")
print(f"Top-5: {top_k_accuracy_ngram(seq, 2, bigram_probs, k=5):.4f}")

Top-1: 0.3333
Top-3: 0.4667
Top-5: 0.6667


In [119]:
# Test on top-k accuracy
print(f"Top-1: {top_k_accuracy_ngram(seq, 3, trigram_probs, k=1):.4f}")
print(f"Top-3: {top_k_accuracy_ngram(seq, 3, trigram_probs, k=3):.4f}")
print(f"Top-5: {top_k_accuracy_ngram(seq, 3, trigram_probs, k=5):.4f}")

Top-1: 0.4286
Top-3: 0.7857
Top-5: 0.8571


## Inference and evaluation with standardized data
Repeat the inference and evaluation using data with only sharp (no flat-sharp duplicates)

In [121]:
# Extract unigrams from df_all
unigram_df = df_all[df_all["ngram_length"] == 1].copy()

# Compute probabilities
unigram_probs = compute_unigram_prob_from_df(unigram_df, alpha=1.0, input_data="standardized")

print(unigram_probs)

bigram_df = df_all[df_all["ngram_length"] == 2].copy()
bigram_probs = compute_ngram_prob_from_df(bigram_df, n=2, alpha=1.0, input_data="standardized")
print(bigram_probs.shape)

trigram_df = df_all[df_all["ngram_length"] == 3].copy()
trigram_probs = compute_ngram_prob_from_df(trigram_df, n=3, alpha=1.0, input_data="standardized")
print(trigram_probs.shape)


         A     Amin        As     Asmin        B      Bmin        C      Cmin  \
  0.092315  0.06682  0.007365  0.001565  0.03589  0.033314  0.13512  0.009891   

        Cs     Csmin  ...        E      Emin         F      Fmin        Fs  \
  0.009358  0.014275  ...  0.06468  0.060952  0.075808  0.007129  0.017186   

     Fsmin         G      Gmin        Gs     Gsmin  
  0.020223  0.157195  0.013375  0.007747  0.007453  

[1 rows x 24 columns]
(24, 24)
(576, 24)


In [122]:
### test inference for bigram ###
seq = []

for _ in range(16):
    if len(seq) == 0:
        evidence = ""
    elif len(seq) == 1:
        evidence = seq[-1]
    else:
        evidence = " ".join(seq[-2:])

    next_chord = probabilistic_inference(evidence) # can change to deterministic_inference()
    seq.append(next_chord)

print(seq)

['G', 'F', 'C', 'E', 'F', 'Fs', 'F', 'Cs', 'Cmin', 'Gs', 'Cs', 'D', 'Cs', 'F', 'G', 'Amin']


In [123]:
# test log-likelihood
song_standardized = ['C', 'F', 'C', 'E', 'Amin', 'C', 'F', 'C', 'G', 'C', 'F', 'C', 'E', 'Amin', 'C', 'F', 'G', 'C', 'F', 'C', 'E', 'Amin', 'C', 'F', 'C', 'G', 'C', 'F', 'C', 'E', 'Amin', 'C', 'F', 'G', 'C', 'F', 'C', 'F', 'C', 'G', 'C', 'F', 'C', 'E', 'Amin', 'C', 'F', 'G', 'C', 'D', 'G', 'D', 'G', 'D', 'A', 'D', 'G', 'D', 'Fs', 'Bmin', 'D', 'G', 'A', 'D', 'G', 'A', 'D']
print(song_log_likelihood_ngram(song_standardized, 1, unigram_probs))
print(song_log_likelihood_ngram(song_standardized, 2, bigram_probs))
print(song_log_likelihood_ngram(song_standardized, 3, trigram_probs))

-151.97528736047866
-119.5641913107009
-101.65983246881055


In [124]:
# Test on top-k accuracy
print(f"Top-1: {top_k_accuracy_ngram(seq, 2, bigram_probs, k=1):.4f}")
print(f"Top-3: {top_k_accuracy_ngram(seq, 2, bigram_probs, k=3):.4f}")
print(f"Top-5: {top_k_accuracy_ngram(seq, 2, bigram_probs, k=5):.4f}")

Top-1: 0.1333
Top-3: 0.2667
Top-5: 0.3333


In [125]:
# Test on top-k accuracy
print(f"Top-1: {top_k_accuracy_ngram(seq, 3, trigram_probs, k=1):.4f}")
print(f"Top-3: {top_k_accuracy_ngram(seq, 3, trigram_probs, k=3):.4f}")
print(f"Top-5: {top_k_accuracy_ngram(seq, 3, trigram_probs, k=5):.4f}")

Top-1: 0.1429
Top-3: 0.3571
Top-5: 0.5000


## Inference and evaluation with augmented data
Repeat the inference and evaluation with the augmented data through transpositions

In [126]:
# Extract unigrams from df_all
unigram_df = df_all[df_all["ngram_length"] == 1].copy()

# Compute probabilities
unigram_probs = compute_unigram_prob_from_df(unigram_df, alpha=1.0, input_data="augmented")

print(unigram_probs)

bigram_df = df_all[df_all["ngram_length"] == 2].copy()
bigram_probs = compute_ngram_prob_from_df(bigram_df, n=2, alpha=1.0, input_data="augmented")
print(bigram_probs.shape)

trigram_df = df_all[df_all["ngram_length"] == 3].copy()
trigram_probs = compute_ngram_prob_from_df(trigram_df, n=3, alpha=1.0, input_data="augmented")
print(trigram_probs.shape)


         A     Amin        As     Asmin        B      Bmin        C      Cmin  \
  0.092315  0.06682  0.007365  0.001565  0.03589  0.033314  0.13512  0.009891   

        Cs     Csmin  ...        E      Emin         F      Fmin        Fs  \
  0.009358  0.014275  ...  0.06468  0.060952  0.075808  0.007129  0.017186   

     Fsmin         G      Gmin        Gs     Gsmin  
  0.020223  0.157195  0.013375  0.007747  0.007453  

[1 rows x 24 columns]
(24, 24)
(576, 24)


In [127]:
### test inference for bigram ###
seq = []

for _ in range(16):
    if len(seq) == 0:
        evidence = ""
    elif len(seq) == 1:
        evidence = seq[-1]
    else:
        evidence = " ".join(seq[-2:])

    next_chord = probabilistic_inference(evidence) # can change to deterministic_inference()
    seq.append(next_chord)

print(seq)

['D', 'C', 'Amin', 'C', 'Amin', 'G', 'F', 'A', 'Dmin', 'As', 'C', 'G', 'Dmin', 'C', 'G', 'Amin']


In [128]:
# test log-likelihood
song_augmented = ['C', 'F', 'C', 'E', 'Amin', 'C', 'F', 'C', 'G', 'C', 'F', 'C', 'E', 'Amin', 'C', 'F', 'G', 'C', 'F', 'C', 'E', 'Amin', 'C', 'F', 'C', 'G', 'C', 'F', 'C', 'E', 'Amin', 'C', 'F', 'G', 'C', 'F', 'C', 'F', 'C', 'G', 'C', 'F', 'C', 'E', 'Amin', 'C', 'F', 'G', 'C', 'D', 'G', 'D', 'G', 'D', 'A', 'D', 'G', 'D', 'Fs', 'Bmin', 'D', 'G', 'A', 'D', 'G', 'A', 'D']
print(song_log_likelihood_ngram(song_augmented, 1, unigram_probs))
print(song_log_likelihood_ngram(song_augmented, 2, bigram_probs))
print(song_log_likelihood_ngram(song_augmented, 3, trigram_probs))

-151.97528736047866
-119.5641913107009
-101.65983246881055


In [129]:
# Test on top-k accuracy
print(f"Top-1: {top_k_accuracy_ngram(seq, 2, bigram_probs, k=1):.4f}")
print(f"Top-3: {top_k_accuracy_ngram(seq, 2, bigram_probs, k=3):.4f}")
print(f"Top-5: {top_k_accuracy_ngram(seq, 2, bigram_probs, k=5):.4f}")

Top-1: 0.2000
Top-3: 0.5333
Top-5: 0.7333


In [130]:
# Test on top-k accuracy
print(f"Top-1: {top_k_accuracy_ngram(seq, 3, trigram_probs, k=1):.4f}")
print(f"Top-3: {top_k_accuracy_ngram(seq, 3, trigram_probs, k=3):.4f}")
print(f"Top-5: {top_k_accuracy_ngram(seq, 3, trigram_probs, k=5):.4f}")

Top-1: 0.2143
Top-3: 0.5714
Top-5: 0.7857
